## Aplicación a caso práctico real del paquete recore 

In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

In [11]:
df = pd.read_csv('com.csv')
carbon_number = df['component'].to_numpy()
molar_fraction = df['molar_percentage'].to_numpy()
molecular_weight = df['molar_mass'].to_numpy()
density = df['density'].to_numpy()

carbonnumber_max = carbon_number[-2]
molar_fraction_max = molar_fraction[-2]
molecular_weight_max = molecular_weight[-2]

df

,component,molar_percentage,molar_mass,density
0,7,2.87,96.0,0.738
1,8,4.08,107.0,0.765
2,9,3.51,121.0,0.781
3,10,3.26,134.0,0.792
4,11,2.51,147.0,0.796
5,12,2.24,161.0,0.810
6,13,2.18,175.0,0.825
7,14,2.07,190.0,0.836
8,15,2.03,206.0,0.842
9,16,1.67,222.0,0.849


Primero ajusto para obtener todas las constantes intervinientes. Para eso utilizo la librería Foo_fit.

Dato: Si no quisiera obtener o chequear los valores de los parámetros de ajuste puedo pasar directamente a la distribución y al no darle a la función un valor para las constantes las calcula automáticamente con una base de datos definida en la clase.

En este ejemplo, por cuestiones de evaluación se calculan y se evidencian

In [12]:
from recore import Foo_fit
foo_fit = Foo_fit()

A,B = foo_fit.fit_AB(carbon_number,molar_fraction)
L,M = foo_fit.fit_LM(carbon_number, density)
Ac, Bc = foo_fit.fit_AcBc(carbon_number, molar_fraction)
Ad = foo_fit.fit_Ad(carbon_number, density)[0]
C = foo_fit.fit_C(carbon_number,molecular_weight) [0]

fit_parameters = {
    'A':A, 'B':B, 'L':L, 'M':M, 'Ac':Ac, 'Bc':Bc
    ,'Ad':Ad,'C':C
}

fit_parameters

{'A': 20.24141696010174,
 'B': -5.350895658855276,
 'L': 0.5181480391330261,
 'M': 0.11698833413234153,
 'Ac': -0.36045285140982836,
 'Bc': 3.415111947477357,
 'Ad': -0.4578051749233114,
 'C': 13.913346938986416}

Para no utilizar los valores establecidos aleatoriamente por Pedersen para el número máximo de carbono
calculo el valor máximo con el criterio de Cismondi et. al. Este método está definido en la librería Residual_fraction.

In [13]:
from recore import Residual_fraction

residual_fraction = Residual_fraction(molecular_weight_max,molar_fraction_max)

carbon_range_it = range(carbonnumber_max,201,1)
res_carbon_number_mx = residual_fraction.carbon_number_max(
    carbon_range_it, Ac, Bc, C
    )

print('Número máximo de carbono calculado:', res_carbon_number_mx)

Número máximo de carbono calculado: 31


/home/marlen/Charapy/Charapy/recore.py:480: RuntimeWarning: invalid value encountered in double_scalars
  (molarfraction_values[:i]*molecularweight_values[:i]).sum()/(


Aplico las funciones de distribución para obtener todas las propiedades extendidas del fluido que se requieren para ingresar luego a la EoS. 

Se realiza para el criterio de Pedersen y para las modificaciones establecidas por Cismondi, cada uno con su correspondiente librería: Distribution_pedersen y Distribution_cismondi.

In [14]:
from recore import Distribution_pedersen, Distribution_cismondi

distribution_pedersen = Distribution_pedersen()
distribution_cismondi = Distribution_cismondi()

c_range = np.arange(carbonnumber_max+1, 81)
ped_density = distribution_pedersen.p_density(c_range,L,M)
cis_density = distribution_cismondi.c_density(c_range, Ad)

ped_mw = distribution_pedersen.p_molecular_weight(c_range)
cis_mw = distribution_cismondi.c_molecular_weight(c_range,C)

ped_mf = distribution_pedersen.p_molar_fraction(c_range,A,B)
cis_mf = distribution_cismondi.c_molar_fraction(c_range, Ac, Bc)

Con este procedimiento ya resolvimos la caracterización de los datos distribuidos. ¿Qué logramos? Distribuir la fracción plus de la muestra de petroleo (aquella fracción que no podemos caracterizar en el laboratorio por la dificultad para separar sus partes), expandir los datos y estamos listos para obtener las propiedades que vamos a utilizar para ingresar a una Ecuación de Estado. 

La distribución para los datos analizados sólo nos permitió expandir óptimamente 3 números de carbono, esto tiene que ver con el criterio de corte en donde establezco que se detenga la distribución cuando se cumple el balance de masa. Esto debe ajustarse con una metodología planteada en el paper de cismondi et. al. y la flexibilidad del paquete me permite realizarlo (es una tarea en la que estoy trabajando que luego modificará la función carbon_number_max)

Ahora simplemente ordeno el set de datos para mostrar el avance

In [15]:
df ['molar_percentage_pedersen'] = df['molar_percentage']
df ['molar_percentage_cismondi'] = df['molar_percentage']
df['molecular_weight_pedersen'] = df['molar_mass']
df['molecular_weight_cismondi'] = df['molar_mass']
df['density_pedersen'] = df['density']
df['density_cismondi'] = df['density']

df = df.drop(['molar_mass','molar_percentage','density'],  axis=1)

In [16]:
df['Tc_pedersen'] = np.nan
df['Tc_cismondi'] = np.nan
df['Pc_pedersen'] = np.nan
df['Pc_cismondi'] = np.nan

### Correlaciones para determinar Tc, Pc, factor acéntrico etc. 

In [17]:
from recore import Correlations

correlations = Correlations('PR')

tc_ped = correlations.critical_temperature(ped_mw,ped_density)
tc_cis = correlations.critical_temperature(cis_mw,cis_density)
pc_ped = correlations.critical_pression(ped_mw, ped_density)
pc_cis = correlations.critical_pression(cis_mw,cis_density)
#afactor_ped = correlations.accentric_factor(ped_mw, ped_density, 'PR')
#afactor_cis = correlations.accentric_factor(cis_mw, cis_density, 'PR')

In [18]:
for i in range (len(c_range)):
    df=df.append(
        {'component': c_range[i],
        'molar_percentage_pedersen': ped_mf[i],
        'molar_percentage_cismondi' : cis_mf[i],
        'molecular_weight_pedersen': ped_mw[i],
        'molecular_weight_cismondi': cis_mw[i],
        'density_pedersen': ped_density[i],
        'density_cismondi': cis_density[i], 
        'Tc_pedersen': tc_ped[i],
        'Tc_cismondi': tc_cis[i],
        'Pc_pedersen': pc_ped[i],
        'Pc_cismondi': pc_cis[i]},
        ignore_index = True)

df

/tmp/ipykernel_1718/1395460101.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(
/tmp/ipykernel_1718/1395460101.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(
/tmp/ipykernel_1718/1395460101.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(
/tmp/ipykernel_1718/1395460101.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(
/tmp/ipykernel_1718/1395460101.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(
/tmp/ipykernel_1718/1395460101.py:2: FutureWarning: The frame.append m

,component,molar_percentage_pedersen,molar_percentage_cismondi,molecular_weight_pedersen,molecular_weight_cismondi,density_pedersen,density_cismondi,Tc_pedersen,Tc_cismondi,Pc_pedersen,Pc_cismondi
0,7.0,2.870000e+00,2.870000e+00,96.0,96.000000,0.738000,0.738000,NaN,NaN,NaN,NaN
1,8.0,4.080000e+00,4.080000e+00,107.0,107.000000,0.765000,0.765000,NaN,NaN,NaN,NaN
2,9.0,3.510000e+00,3.510000e+00,121.0,121.000000,0.781000,0.781000,NaN,NaN,NaN,NaN
3,10.0,3.260000e+00,3.260000e+00,134.0,134.000000,0.792000,0.792000,NaN,NaN,NaN,NaN
4,11.0,2.510000e+00,2.510000e+00,147.0,147.000000,0.796000,0.796000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
70,76.0,1.503270e-168,3.854076e-11,1060.0,1057.934286,1.024793,0.936020,-520.811271,-527.517540,70.818005,1.902461
71,77.0,7.131361e-171,2.687680e-11,1074.0,1071.847633,1.026323,0.936042,-519.421651,-526.243968,82.392828,1.899279
72,78.0,3.383047e-173,1.874282e-11,1088.0,1085.760980,1.027832,0.936061,-518.050023,-524.986953,96.199763,1.896150
73,79.0,1.604884e-175,1.307050e-11,1102.0,1099.674327,1.029322,0.936079,-516.695928,-523.746063,112.721712,1.893073


In [18]:
# C1 a C10 y de C11 a C20
la_tupla = [(0, 9), (10, 19)]

for i in la_tupla:
    print (i)



(0, 9)
(10, 19)


In [22]:
la_tupla[1][0]

10